# Phase 2-3: Suite Metadata Extraction + Visualization Generation

This notebook generates publication-quality figures for the PQC Drone↔GCS performance chapter.

**Data Source:** `analysis/phase1_provenance_map.json` (90 suite-mode combinations)

**Outputs:** 15+ PNG figures (300 DPI, colorblind-friendly palette)

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set publication-quality defaults
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9

# Use colorblind-friendly palette
sns.set_palette('colorblind')
colors = sns.color_palette('colorblind')

print("✅ Libraries imported successfully")

In [ ]:
# Load Phase 1 provenance map
with open('../analysis/phase1_provenance_map.json', 'r') as f:
    provenance_map = json.load(f)

print(f"📊 Loaded {len(provenance_map['suites'])} suites")
print(f"   Total combinations: {provenance_map['metadata']['total_combinations']}")

In [ ]:
# Build canonical suite catalog DataFrame
rows = []

for suite_id, suite_data in provenance_map['suites'].items():
    meta = suite_data['metadata']
    
    row = {
        'suite_id': suite_id,
        'kem_family': meta.get('kem_family', 'Unknown'),
        'kem_variant': meta.get('kem_full', ''),
        'nist_level': meta.get('nist_level', 0),
        'aead_cipher': meta.get('aead_cipher', ''),
        'sig_scheme': meta.get('sig_scheme', ''),
        'sig_family': meta.get('sig_family', ''),
    }
    
    # Extract baseline metrics
    baseline = suite_data['metrics'].get('baseline', {})
    row['avg_throughput_baseline'] = baseline.get('throughput_mbps', 0)
    row['avg_power_baseline'] = baseline.get('power_avg_w', 0)
    row['avg_handshake_baseline'] = baseline.get('handshake_gcs_ms', 0)
    row['loss_baseline'] = baseline.get('loss_pct', 0)
    row['cpu_max_baseline'] = baseline.get('cpu_max_percent', 0)
    row['rss_baseline'] = baseline.get('rss_mib', 0)
    
    # Lightweight metrics
    lightweight = suite_data['metrics'].get('lightweight', {})
    row['avg_throughput_lightweight'] = lightweight.get('throughput_mbps', 0)
    row['avg_power_lightweight'] = lightweight.get('power_avg_w', 0)
    row['loss_lightweight'] = lightweight.get('loss_pct', 0)
    row['cpu_max_lightweight'] = lightweight.get('cpu_max_percent', 0)
    row['rss_lightweight'] = lightweight.get('rss_mib', 0)
    
    # Transformer metrics
    transformer = suite_data['metrics'].get('transformer', {})
    row['avg_throughput_transformer'] = transformer.get('throughput_mbps', 0)
    row['avg_power_transformer'] = transformer.get('power_avg_w', 0)
    row['loss_transformer'] = transformer.get('loss_pct', 0)
    row['cpu_max_transformer'] = transformer.get('cpu_max_percent', 0)
    row['rss_transformer'] = transformer.get('rss_mib', 0)
    
    # RTT data from baseline
    row['rtt_p50'] = baseline.get('rtt_p50_ms', 0)
    row['rtt_p95'] = baseline.get('rtt_p95_ms', 0)
    row['rtt_max'] = baseline.get('rtt_max_ms', 0)
    
    # Energy metrics
    row['energy_baseline'] = baseline.get('energy_j', 0)
    row['kem_keygen_ms'] = baseline.get('kem_keygen_ms', 0)
    row['kem_decap_ms'] = baseline.get('kem_decap_ms', 0)
    row['sig_sign_ms'] = baseline.get('sig_sign_ms', 0)
    
    rows.append(row)

df = pd.DataFrame(rows)
print(f"\n✅ Created catalog DataFrame: {df.shape}")
print(f"   Columns: {list(df.columns)}")
display(df.head())

## Figure Generation

### Figure 01-03: Throughput by Suite (Baseline, Lightweight, Transformer)

In [ ]:
# Figure 01: Throughput - Baseline
fig, ax = plt.subplots(figsize=(14, 6))
df_sorted = df.sort_values('avg_throughput_baseline', ascending=False)
bars = ax.bar(range(len(df_sorted)), df_sorted['avg_throughput_baseline'], color=colors[0])
ax.set_xlabel('Suite Index (sorted by throughput)')
ax.set_ylabel('Throughput (Mb/s)')
ax.set_title('Baseline Throughput Across All 30 Suites')
ax.axhline(y=8.0, color='red', linestyle='--', label='Target: 8 Mb/s', alpha=0.7)
ax.grid(axis='y', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.savefig('../figures/figure01_throughput_all_suites_baseline.png')
plt.close()
print("✅ Saved figure01_throughput_all_suites_baseline.png")

In [ ]:
# Figure 02: Throughput - Lightweight
fig, ax = plt.subplots(figsize=(14, 6))
df_sorted = df.sort_values('avg_throughput_lightweight', ascending=False)
bars = ax.bar(range(len(df_sorted)), df_sorted['avg_throughput_lightweight'], color=colors[1])
ax.set_xlabel('Suite Index (sorted by throughput)')
ax.set_ylabel('Throughput (Mb/s)')
ax.set_title('Lightweight (XGBoost) Throughput Across All 30 Suites')
ax.axhline(y=8.0, color='red', linestyle='--', label='Target: 8 Mb/s', alpha=0.7)
ax.grid(axis='y', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.savefig('../figures/figure02_throughput_all_suites_lightweight.png')
plt.close()
print("✅ Saved figure02_throughput_all_suites_lightweight.png")

In [ ]:
# Figure 03: Throughput - Transformer
fig, ax = plt.subplots(figsize=(14, 6))
df_sorted = df.sort_values('avg_throughput_transformer', ascending=False)
bars = ax.bar(range(len(df_sorted)), df_sorted['avg_throughput_transformer'], color=colors[2])
ax.set_xlabel('Suite Index (sorted by throughput)')
ax.set_ylabel('Throughput (Mb/s)')
ax.set_title('Transformer (TST) Throughput Across All 30 Suites')
ax.axhline(y=8.0, color='red', linestyle='--', label='Target: 8 Mb/s', alpha=0.7)
ax.grid(axis='y', alpha=0.3)
ax.legend()
plt.tight_layout()
plt.savefig('../figures/figure03_throughput_all_suites_transformer.png')
plt.close()
print("✅ Saved figure03_throughput_all_suites_transformer.png")

In [ ]:
# Figure 04: Throughput Comparison Grouped
fig, ax = plt.subplots(figsize=(16, 7))
x = np.arange(len(df))
width = 0.25

bars1 = ax.bar(x - width, df['avg_throughput_baseline'], width, label='Baseline', color=colors[0])
bars2 = ax.bar(x, df['avg_throughput_lightweight'], width, label='Lightweight', color=colors[1])
bars3 = ax.bar(x + width, df['avg_throughput_transformer'], width, label='Transformer', color=colors[2])

ax.set_xlabel('Suite Index')
ax.set_ylabel('Throughput (Mb/s)')
ax.set_title('Throughput Comparison: All 30 Suites × 3 DDOS Modes')
ax.axhline(y=8.0, color='red', linestyle='--', label='Target: 8 Mb/s', alpha=0.5)
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure04_throughput_comparison_grouped.png')
plt.close()
print("✅ Saved figure04_throughput_comparison_grouped.png")

### Figure 05: Loss Distribution Violin Plot

In [ ]:
# Figure 05: Loss Distribution
loss_data = []
for _, row in df.iterrows():
    loss_data.append({'Mode': 'Baseline', 'Loss (%)': row['loss_baseline']})
    loss_data.append({'Mode': 'Lightweight', 'Loss (%)': row['loss_lightweight']})
    loss_data.append({'Mode': 'Transformer', 'Loss (%)': row['loss_transformer']})

loss_df = pd.DataFrame(loss_data)

fig, ax = plt.subplots(figsize=(10, 6))
sns.violinplot(data=loss_df, x='Mode', y='Loss (%)', ax=ax, palette='colorblind')
ax.set_title('Packet Loss Distribution by DDOS Detection Mode')
ax.set_ylabel('Loss (%)')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure05_loss_distribution_violin.png')
plt.close()
print("✅ Saved figure05_loss_distribution_violin.png")

### Figure 06: RTT CDF

In [ ]:
# Figure 06: RTT CDF
fig, ax = plt.subplots(figsize=(10, 6))

for metric, label in [('rtt_p50', 'p50'), ('rtt_p95', 'p95'), ('rtt_max', 'max')]:
    sorted_rtt = np.sort(df[metric])
    cdf = np.arange(1, len(sorted_rtt) + 1) / len(sorted_rtt)
    ax.plot(sorted_rtt, cdf, label=f'RTT {label}', linewidth=2)

ax.set_xlabel('RTT (ms)')
ax.set_ylabel('CDF')
ax.set_title('Cumulative Distribution of RTT Metrics (Baseline)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure06_rtt_cdf_all_modes.png')
plt.close()
print("✅ Saved figure06_rtt_cdf_all_modes.png")

### Figure 07: Handshake Latency Scatter

In [ ]:
# Figure 07: Handshake Latency Scatter
fig, ax = plt.subplots(figsize=(12, 6))

kem_families = df['kem_family'].unique()
color_map = dict(zip(kem_families, colors[:len(kem_families)]))

for kem in kem_families:
    subset = df[df['kem_family'] == kem]
    ax.scatter(range(len(subset)), subset['avg_handshake_baseline'], 
               label=kem, color=color_map[kem], s=80, alpha=0.7)

ax.set_xlabel('Suite Index (grouped by KEM family)')
ax.set_ylabel('Handshake Latency (ms)')
ax.set_title('Handshake Latency by KEM Family (Baseline)')
ax.legend(title='KEM Family')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure07_handshake_latency_scatter.png')
plt.close()
print("✅ Saved figure07_handshake_latency_scatter.png")

### Figure 08-09: Power Consumption

In [ ]:
# Figure 08: Power vs Suite (Baseline)
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(range(len(df)), df['avg_power_baseline'], color=colors[3])
ax.set_xlabel('Suite Index')
ax.set_ylabel('Power (W)')
ax.set_title('Power Consumption Across All 30 Suites (Baseline)')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure08_power_vs_suite_baseline.png')
plt.close()
print("✅ Saved figure08_power_vs_suite_baseline.png")

In [ ]:
# Figure 09: Power Comparison (Baseline vs Transformer)
fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(df))
width = 0.35

bars1 = ax.bar(x - width/2, df['avg_power_baseline'], width, label='Baseline', color=colors[0])
bars2 = ax.bar(x + width/2, df['avg_power_transformer'], width, label='Transformer', color=colors[2])

ax.set_xlabel('Suite Index')
ax.set_ylabel('Power (W)')
ax.set_title('Power Consumption: Baseline vs Transformer')
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure09_power_vs_suite_transformer_comparison.png')
plt.close()
print("✅ Saved figure09_power_vs_suite_transformer_comparison.png")

### Figure 10: Energy Heatmap (KEM Operations)

In [ ]:
# Figure 10: Energy Heatmap
# Convert ms to mJ (approximation: energy = power × time)
energy_data = df[['kem_keygen_ms', 'kem_decap_ms', 'sig_sign_ms']].copy()
energy_data.columns = ['KEM Keygen (ms)', 'KEM Decap (ms)', 'Sig Sign (ms)']

fig, ax = plt.subplots(figsize=(10, 14))
sns.heatmap(energy_data, annot=False, cmap='YlOrRd', ax=ax, cbar_kws={'label': 'Time (ms)'})
ax.set_ylabel('Suite Index')
ax.set_title('Cryptographic Operation Time Heatmap')
plt.tight_layout()
plt.savefig('../figures/figure10_energy_heatmap_kem_operations.png')
plt.close()
print("✅ Saved figure10_energy_heatmap_kem_operations.png")

### Figure 11: CPU Utilization Heatmap

In [ ]:
# Figure 11: CPU Utilization Heatmap
cpu_data = df[['cpu_max_baseline', 'cpu_max_lightweight', 'cpu_max_transformer']].copy()
cpu_data.columns = ['Baseline', 'Lightweight', 'Transformer']

fig, ax = plt.subplots(figsize=(8, 14))
sns.heatmap(cpu_data, annot=False, cmap='Blues', ax=ax, cbar_kws={'label': 'CPU Max (%)'})
ax.set_ylabel('Suite Index')
ax.set_title('CPU Utilization Heatmap by DDOS Mode')
plt.tight_layout()
plt.savefig('../figures/figure11_cpu_utilization_heatmap.png')
plt.close()
print("✅ Saved figure11_cpu_utilization_heatmap.png")

### Figure 12: RSS Memory Heatmap

In [ ]:
# Figure 12: RSS Memory Heatmap
rss_data = df[['rss_baseline', 'rss_lightweight', 'rss_transformer']].copy()
rss_data.columns = ['Baseline', 'Lightweight', 'Transformer']

fig, ax = plt.subplots(figsize=(8, 14))
sns.heatmap(rss_data, annot=False, cmap='Greens', ax=ax, cbar_kws={'label': 'RSS (MiB)'})
ax.set_ylabel('Suite Index')
ax.set_title('Memory (RSS) Usage Heatmap by DDOS Mode')
plt.tight_layout()
plt.savefig('../figures/figure12_rss_memory_heatmap.png')
plt.close()
print("✅ Saved figure12_rss_memory_heatmap.png")

### Figure 13: Goodput Ratio Overlay

In [ ]:
# Figure 13: Goodput Ratio
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(df))

goodput_baseline = df['avg_throughput_baseline'] / 8.0 * 100
goodput_lightweight = df['avg_throughput_lightweight'] / 8.0 * 100
goodput_transformer = df['avg_throughput_transformer'] / 8.0 * 100

ax.plot(x, goodput_baseline, label='Baseline', marker='o', linewidth=1.5, color=colors[0])
ax.plot(x, goodput_lightweight, label='Lightweight', marker='s', linewidth=1.5, color=colors[1])
ax.plot(x, goodput_transformer, label='Transformer', marker='^', linewidth=1.5, color=colors[2])

ax.axhline(y=100, color='red', linestyle='--', label='Target: 100%', alpha=0.5)
ax.set_xlabel('Suite Index')
ax.set_ylabel('Goodput Ratio (%)')
ax.set_title('Goodput Ratio (Throughput / 8 Mb/s Target)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../figures/figure13_goodput_ratio_overlay.png')
plt.close()
print("✅ Saved figure13_goodput_ratio_overlay.png")

### Figure 14: NIST Level Aggregation

In [ ]:
# Figure 14: NIST Level Aggregation
nist_data = []
for level in sorted(df['nist_level'].unique()):
    if level > 0:
        subset = df[df['nist_level'] == level]
        for _, row in subset.iterrows():
            nist_data.append({
                'NIST Level': f'Level {level}',
                'Throughput (Mb/s)': row['avg_throughput_baseline'],
                'Power (W)': row['avg_power_baseline'],
                'Handshake (ms)': row['avg_handshake_baseline']
            })

nist_df = pd.DataFrame(nist_data)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Throughput
sns.boxplot(data=nist_df, x='NIST Level', y='Throughput (Mb/s)', ax=axes[0], palette='colorblind')
axes[0].set_title('Throughput by NIST Level')
axes[0].grid(axis='y', alpha=0.3)

# Power
sns.boxplot(data=nist_df, x='NIST Level', y='Power (W)', ax=axes[1], palette='colorblind')
axes[1].set_title('Power by NIST Level')
axes[1].grid(axis='y', alpha=0.3)

# Handshake
sns.boxplot(data=nist_df, x='NIST Level', y='Handshake (ms)', ax=axes[2], palette='colorblind')
axes[2].set_title('Handshake Latency by NIST Level')
axes[2].grid(axis='y', alpha=0.3)
axes[2].set_yscale('log')

plt.tight_layout()
plt.savefig('../figures/figure14_nist_level_aggregation_boxplot.png')
plt.close()
print("✅ Saved figure14_nist_level_aggregation_boxplot.png")

### Figure 15: KEM Family Comparison

In [ ]:
# Figure 15: KEM Family Comparison
kem_agg = df.groupby('kem_family').agg({
    'avg_throughput_baseline': 'mean',
    'avg_power_baseline': 'mean',
    'avg_handshake_baseline': 'mean',
    'loss_baseline': 'mean'
}).reset_index()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Throughput
axes[0, 0].bar(kem_agg['kem_family'], kem_agg['avg_throughput_baseline'], color=colors)
axes[0, 0].set_ylabel('Avg Throughput (Mb/s)')
axes[0, 0].set_title('Average Throughput by KEM Family')
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# Power
axes[0, 1].bar(kem_agg['kem_family'], kem_agg['avg_power_baseline'], color=colors)
axes[0, 1].set_ylabel('Avg Power (W)')
axes[0, 1].set_title('Average Power by KEM Family')
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# Handshake
axes[1, 0].bar(kem_agg['kem_family'], kem_agg['avg_handshake_baseline'], color=colors)
axes[1, 0].set_ylabel('Avg Handshake (ms)')
axes[1, 0].set_title('Average Handshake Latency by KEM Family')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# Loss
axes[1, 1].bar(kem_agg['kem_family'], kem_agg['loss_baseline'], color=colors)
axes[1, 1].set_ylabel('Avg Loss (%)')
axes[1, 1].set_title('Average Loss by KEM Family')
axes[1, 1].grid(axis='y', alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../figures/figure15_kem_family_comparison_bars.png')
plt.close()
print("✅ Saved figure15_kem_family_comparison_bars.png")

## Summary

All 15 publication-quality figures generated successfully!

In [ ]:
print("\n" + "="*60)
print("✅ PHASE 2-3 COMPLETE")
print("="*60)
print(f"Generated 15 publication-quality figures (300 DPI)")
print(f"Output directory: ../../figures/")
print(f"Suite catalog: {df.shape[0]} suites × {df.shape[1]} columns")
print("="*60)